In [1]:
import tensorflow as tf
import numpy as np
from trade_sim import Market
import tensorflow.contrib as contrib
import os
from multiprocessing import Process, Queue
import random

In [9]:
class Agent:
    def __init__(self, numberOfCurrencies, timeFrame, sess, initialPortfolio=10000.0):
        self._s = sess
        self.inputT = tf.placeholder(shape=[None, numberOfCurrencies, timeFrame, 3], dtype=tf.float32)
        self.conv1 = tf.layers.conv2d(inputs=self.inputT, filters=2, kernel_size=[1,3], activation=tf.nn.relu)
      #  self.conv1 = tf.nn.depthwise_conv2d(self.inputT, [1,3,3,4], [1,1,1,1], 'SAME')
        #self.conv2 = tf.layers.conv2d(inputs=self.conv1, filters=150, kernel_size=[1,8], activation=tf.nn.relu)
        #self.conv3 = tf.layers.conv2d(inputs = self.conv2, filters=200, kernel_size=[1,41], activation=tf.nn.relu)
        #self.conv4 = tf.layers.conv2d(inputs=self.conv3, filters=30, kernel_size=[1,1] , activation=tf.nn.relu)
        self.hiddenMid = tf.layers.dense(self.conv1, 1000, activation=tf.nn.relu)
        self.conv2 = tf.layers.conv2d(inputs=self.hiddenMid, filters=30, kernel_size=[1,48], activation=tf.nn.relu)
        self.final = tf.layers.dense(self.conv2, 20000, activation=tf.nn.relu)
        self.hidden0 = tf.layers.dense(self.final, 1000, activation=tf.nn.relu)
        self.hidden = tf.layers.dense(self.hidden0, 2000, activation=tf.nn.relu)
        self.final2 = tf.layers.dense(self.hidden, 1)
        self._allocate = tf.nn.softmax(self.final2, axis=1)
        
        self.priceChanges = tf.placeholder(shape=[None, numberOfCurrencies, 1], dtype=tf.float32)
        
        #self.loss = -tf.matmul(tf.matrix_transpose(tf.nn.leaky_relu(tf.log(self.priceChanges), alpha=10)),tf.reshape(self._allocate, [-1, numberOfCurrencies, 1]))
        self.averageLoss = tf.reduce_mean(tf.matmul(tf.matrix_transpose(self.priceChanges), 
                                             tf.scalar_mul(tf.constant(initialPortfolio), 
                                               tf.reshape(self._allocate, [-1, numberOfCurrencies, 1]))))

        self.loss = tf.exp(tf.reduce_sum(tf.multiply(-tf.log(self.priceChanges), tf.reshape(self._allocate, [tf.shape(self._allocate)[0], numberOfCurrencies, 1]))))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
        self._train = self.optimizer.minimize(self.loss)
        
    def act(self, observation):
        return self._s.run(self._allocate, feed_dict={self.inputT: observation})
    
    def train_step(self, obs, prices):
        batch_feed = {self.inputT : obs,
                     self.priceChanges: prices
                     }
        _, lossValue = self._s.run([self._train, self.averageLoss], feed_dict=batch_feed)
        return lossValue
    def test_model(self, obs, prices):
        print(self._s.run(self.averageLoss, feed_dict={self.inputT : obs, self.priceChanges: prices}))
        print(prices)

In [10]:
def main():
    feed = feedBackup.copy()
    seeds = [3, 5, 7]
    tf.reset_default_graph()
    with tf.Session() as sess:
        tf.set_random_seed(seeds[1])
        test1 = Agent(len(testSim.currencies), 50, sess)
        sess.run(tf.global_variables_initializer())
        
        testData = feed[0]
        feed.pop(0)
        
        
            
        try:
            for episode in range(10000):
                print("Episode: {}".format(episode))
                index = list(range(len(feed)))
                loss = 0
                count = len(feed)
                while len(index) != 0:
                    for i in random.sample(index, 1):
                        with tf.device('/gpu:0'):
                            loss += test1.train_step(feed[i][0], feed[i][1])
                        index.remove(i)
                print(loss/count)
        except KeyboardInterrupt:
            print("Ending")
            test1.test_model(testData[0], testData[1])
            
if __name__ == "__main__":
    main()

Episode: 0
10000.002197265625
Episode: 1
10000.002197265625
Episode: 2
10000.001953125
Episode: 3
10000.001953125
Episode: 4
10000.00244140625
Episode: 5
10000.002197265625
Episode: 6
10000.001953125
Episode: 7
10000.001953125
Episode: 8
10000.001953125
Episode: 9
10000.002197265625
Episode: 10
10000.001953125
Episode: 11
10000.001953125
Episode: 12
10000.001953125
Episode: 13
10000.001953125
Episode: 14
10000.00146484375
Episode: 15
10000.001953125
Episode: 16
10000.001953125
Episode: 17
10000.001708984375
Episode: 18
10000.001953125
Episode: 19
10000.001953125
Episode: 20
10000.002197265625
Episode: 21
10000.001953125
Episode: 22
10000.001953125
Episode: 23
10000.001953125
Episode: 24
10000.002197265625
Episode: 25
10000.001953125
Episode: 26
10000.001953125
Episode: 27
10000.001953125
Episode: 28
10000.002197265625
Episode: 29
10000.001953125
Episode: 30
10000.00244140625
Episode: 31
10000.002197265625
Episode: 32
10000.001708984375
Episode: 33
10000.001953125
Episode: 34
Ending
100

In [6]:
def importData(simulator):
    testSim = simulator
    q = Queue()
    jobs = []
    PERIOD_SIZE = 50
    BATCH_SIZE = 250
    BATCH_COUNT = 5
    BATCH_OFFSET = 10
    dates = testSim.getAllDates()
    index = list(range(BATCH_COUNT))
    feed = []
    threads = 16

    running = False
    count = 0
    while 1:
        if count < threads:
            for i in random.sample(index, threads-count if len(index) >= threads-count else len(index)):
                p = Process(target=testSim.processTimePeriod, args=(q, PERIOD_SIZE, dates, BATCH_SIZE * (i + BATCH_OFFSET) + PERIOD_SIZE, BATCH_SIZE))
                jobs.append(p)
                print("Starting")
                p.start() 
                index.remove(i)
        count = 0
        for p in jobs:
            if not p.is_alive():
                p.terminate()
                jobs.remove(p)
            else:
                count += 1
        while not q.empty():
            print('Getting')
            feed.append(q.get())
        if count == 0 and len(index) == 0 and len(feed) == BATCH_COUNT:
            print('empty')
            break
    print(len(feed))
    return feed

In [7]:
feedBackup = importData(testSim)

Starting
Starting
Starting
Starting
Starting
Done
Done
Getting
Getting
Done
Done
Getting
Getting
Getting
Done
empty
5


In [4]:
testSim = Market(['EUR','USD', 'TRY', 'JPY'], os.path.abspath('../Data_Processing/ProcessedData'))